In [1]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
from elasticsearch_dsl import Search
from datetime import datetime
from tqdm import tqdm
import json

In [2]:
today = datetime.now().strftime('%Y-%m-%d')

languages = ['in', 'en']

In [3]:
for lang in languages:
    es = Elasticsearch(
        hosts = [{'host': 'localhost', 'port': 9200}],
        connection_class = RequestsHttpConnection,
    )
    s = Search(using=es, index='twitter_raw').filter("term", lang__keyword = lang)\
    .filter('range', datetime = {'gte': '2020-02-22', 'lte': today, 'format': 'yyyy-MM-dd'})\
    .source(['data_text', 'retweet_text_full'])
    results = []
    for hit in tqdm(s.scan()):
        results.append(hit.to_dict())
        
    print(lang, len(results))
    
    with open(f'{today}-twitter-dump-{lang}.json', 'w') as fopen:
        json.dump(results, fopen)

1711555it [09:09, 3113.56it/s]


in 1711555


716237it [03:35, 3329.75it/s]


en 716237


In [4]:
from glob import glob

files = glob('*-twitter*.json')
files

['2020-03-08-twitter-dump-in.json', '2020-03-08-twitter-dump-en.json']

In [5]:
import boto3

bucketName = 'malaya-dataset'

for file in files:
    outPutname = f"dumping/twitter/{file}"

    s3 = boto3.client('s3')
    s3.upload_file(file,bucketName,outPutname)